In [1]:
import numpy as np
import geopandas as gpd


import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im
import scipy.optimize as opt

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp

from fsspec.implementations.local import LocalFileSystem

In [2]:
from dask.distributed import Client, LocalCluster
#
cluster = LocalCluster()
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster(processes=5, cores=5,walltime='02:00:00') #processes=7, cores=7
#w = cluster.scale(jobs=10)
#
client = Client(cluster)

In [3]:
# Load Lagrangian data
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'

In [4]:
parcels_index = 'lpass_filtered_05cpd'
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df.head()


,trajectory,lat,lon,z,zonal_velocity,meridional_velocity,waterdepth,zonal_velocity_hat_real,zonal_velocity_hat_imag,meridional_velocity_hat_real,meridional_velocity_hat_imag
time,,,,,,,,,,,
2011-11-15,2000844,-67.226303,0.562500,0.0,0.0,0.0,0.0,-0.037399,0.0,-0.038937,0.0
2011-11-15,2001062,-68.788177,10.979167,0.0,0.0,0.0,0.0,-0.050828,0.0,-0.055424,0.0
2011-11-15,2001076,-62.904213,10.979167,0.0,0.0,0.0,0.0,-0.052928,0.0,-0.013808,0.0
2011-11-15,3001600,-60.772732,31.812500,0.0,0.0,0.0,0.0,0.026066,0.0,-0.010228,0.0
2011-11-15,4000298,-28.264765,-23.395834,0.0,0.0,0.0,0.0,-0.037163,0.0,0.025803,0.0


In [ ]:
df['zonal_velocity_energy'] = df['zonal_velocity_hat_real'].apply(lambda x : x**2)

In [ ]:
_df = df[['zonal_velocity_energy','lon','lat','waterdepth','trajectory']]
_df.head()

### Load Eulerian dataset

In [ ]:
ds_low = xr.open_zarr('/home/datawork-lops-osi/equinox/mit4320/diags/Eulerian_lf.zarr')
ds_low

### Binning

In [ ]:
dl = 2.
lon_bins = np.arange(-180.,180., dl)
lat_bins = np.arange(-60, 60, dl)

In [ ]:
N = histogram(ds_low.lon.rename('lon'), ds_low.lat.rename('lat'), 
              bins=[lon_bins,lat_bins], 
              dim=['i', 'j', 'face']
             ).persist()

In [ ]:
ds_binned = (histogram(ds_low.lon.rename('lon'), ds_low.lat.rename('lat'),bins=[lon_bins, lat_bins],
                       weights=ds_low.energy.fillna(0.),
                       dim=['i', 'j', 'face'])/(N)).rename('E_energy').persist()

In [ ]:
def binning(_df,binsize):
    #lon_bins,lat_bins = np.arange(lonmin,lonmax,binsize),np.arange(latmin,latmax,binsize)
    x_idx,y_idx = pd.IntervalIndex.from_breaks(lon_bins),pd.IntervalIndex.from_breaks(lat_bins)
    _df['lon_cut'] = _df['lon'].map_partitions(pd.cut, bins=lon_bins)
    _df['lat_cut'] = _df['lat'].map_partitions(pd.cut, bins=lat_bins)
    _df = _df.drop(columns=['lon','lat','trajectory']) # ,'h'
    return _df
df = binning(df,dl).persist()

In [ ]:
df_mean = df.groupby(['lon_cut','lat_cut']).mean()
df_mean = df_mean.reset_index().persist()

In [ ]:
df_mean['lon_bins'] = df_mean.lon_cut.apply(lambda x :x.mid)
df_mean['lat_bins'] = df_mean.lat_cut.apply(lambda x :x.mid)
df_mean = df_mean.drop(columns=['lon_cut','lat_cut']).persist()
df_mean.head()

In [ ]:
df_mean = df_mean.compute()

In [ ]:
dsL = df_mean.set_index(['lon_bins','lat_bins']).to_xarray().unstack()

In [7]:
cluster.close()

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/home1/